Сделаем предобработку данных: 
 
* Объединим общие данные о фильмах [tmdb_5000_movies](https://files.sberdisk.ru/s/te4QbzdxKgsFQXA) и каст фильмов 
[tmdb_5000_credits](https://files.sberdisk.ru/s/H9oRuXQt5mFz3T9). 
* Оставим в датасете только фильмы, которые вышли в "релиз".
* Уберем фильмы с пропусками в колонках ['overview', 'genres', 'keywords']
* Выведем количество фильмов, оставшихся в выборке 

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
movies = pd.read_csv('../../datasets/tmdb_5000_movies.csv')
credits = pd.read_csv('../../datasets/tmdb_5000_credits.csv')
new_movies = pd.merge(movies, credits, how='left', left_on='id', right_on='movie_id')
new_movies = new_movies[new_movies['status'] == 'Released'].dropna(subset=['overview', 'genres', 'keywords'])
new_movies.rename(columns={'title_x': 'title'}, inplace=True)
new_movies.drop(['movie_id', 'title_y'], axis=1, inplace=True)
new_movies['release_year'] = new_movies['release_date'].str[:4]

print(f'Количество фильмов, оставшихся после фильтрации = {new_movies.shape[0]}')

Количество фильмов, оставшихся после фильтрации = 4792


Реализуем алгоритм рекомендации на основе описания фильма (`overview`) и ключевых слов к фильму (`keywords`). 
Объединим тексты этих колонок и проведем предобработку:
* NaN в описании фильма заменены на пустой символ `''`
* Удалены все английские стоп-слова (используй параметр `stop_words` в `TfidfVectorizer`)
* Рассчитана матрицу [Tf-Idf](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) для описания фильмов(`max_features` в `TfidfVectorizer` равен 10000)

In [4]:
new_movies['overview'] = new_movies['overview'].fillna('')
new_movies['overview_and_keywords'] = new_movies[['overview', 'keywords']].agg(' '.join, axis=1)
tfidf = TfidfVectorizer(stop_words='english', max_features=10000)
movies_tfidf = tfidf.fit_transform(new_movies['overview_and_keywords'])
print(f'Размер получившейся матрицы Tf-Idf = {movies_tfidf.shape}')

Размер получившейся матрицы Tf-Idf = (4792, 10000)


Рассчитаем [cosine similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) 
между фильмами. Составим из этой матрицы датафрейм. 

Сохраним получившийся датафрейм c расстояниями в папку [assets](src/assets) с названием `distance.csv`.
А сам объединенный датасет с фильмами сохраним в папку [assets](src/assets) с названием `movies.csv`.

In [5]:
cosine_sim = linear_kernel(movies_tfidf)
distance = pd.DataFrame(cosine_sim, index=new_movies.id, columns=new_movies.id)
# distance.columns = [str(x) for x in distance.columns]

In [6]:
distance.head(2)

id,19995,285,206647,49026,49529,559,38757,99861,767,209112,...,182291,286939,124606,14337,67238,9367,72766,231617,126186,25975
id,,,,,,,,,,,,,,,,,,,,,
19995,1.0000,0.1616,0.084173,0.141075,0.290498,0.163335,0.109851,0.135357,0.096274,0.092502,...,0.127258,0.0,0.062076,0.114813,0.0,0.043521,0.0,0.060299,0.010252,0.040575
285,0.1616,1.0000,0.079563,0.134989,0.148862,0.155402,0.101430,0.095819,0.091001,0.078926,...,0.120289,0.0,0.056188,0.085687,0.0,0.048129,0.0,0.067340,0.000000,0.038353


In [7]:
distance.to_csv('../assets/distance.csv')
new_movies.to_csv('../assets/movie.csv')